# Template

This notebook allows users to find common and specific genes in their experiment of interest using an *existing* VAE model and selecting a template experiment that is included in the training compendium.

This notebook will generate a `generic_gene_summary_<experiment id>.tsv` file that contains z-scores per gene that indicates how specific a gene is the experiment in question.

In [1]:
%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

In [2]:
import os
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from ponyo import utils, simulate_expression_data
from sophie import (
    process,
    stats,
    ranking,
)

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/matplotlib/__init__.py:886: MatplotlibDeprecationWarning: 
examples.directory is deprecated; in the future, examples will be found relative to the 'datapath' directory.
  "found relative to the 'datapath' directory.".format(key))
Using TensorFlow backend.


## Inputs

User needs to fill in the [config file](config_new_experiment.tsv) following the instructions provided in the [readme]](README.md)

In [3]:
# Read in config variables
config_filename = "config_example.tsv"

params = utils.read_config(config_filename)

In [4]:
# Load config params

# Root directory containing analysis subdirectories and scripts
base_dir = params["base_dir"]

# Local directory to store intermediate files
local_dir = params["local_dir"]

# Normalized compendium filename
normalized_compendium_filename = params["normalized_compendium_filename"]

# Metadata file that maps experiment ids to sample ids
metadata_filename = params["metadata_filename"]

# Delimiter used in metadata file
metadata_delimiter = params["metadata_delimiter"]

# Column header corresponding to experiment id and sample id
metadata_experiment_colname = params["metadata_experiment_colname"]
metadata_sample_colname = params["metadata_sample_colname"]

# ID for template experiment to be selected
project_id = params["project_id"]

# Number of simulated experiments to generate
num_simulated = params["num_simulated"]

# Directory containing simulated experiments
simulated_data_dir = params["simulated_data_dir"]

# Size of the latent dimension
latent_dim = params["latent_dim"]

# Scaler transform used to scale compendium data into 0-1 range for training
## TO DO: change to scaler_filename with new version
scaler_transform_filename = params["scaler_transform_filename"]

# Which DE method to use
# We recommend that if data is RNA-seq then use DESeq2
# If data is microarray then use Limma
de_method = params["DE_method"]

# If using DE-seq, setting this parameter will
# remove genes below a certain threshold
count_threshold = params["count_threshold"]

# Metadata file that specifies which samples to keep for DE analysis (Optional)
template_process_samples_filename = params["template_process_samples_filename"]
    
# Metadata file that specifies sample grouping for DE analysis
template_DE_grouping_filename = params["template_DE_grouping_filename"]

# Statistic to use to rank genes or pathways by
# Choices are {} FILL IN
col_to_rank_genes = params["rank_genes_by"]

In [5]:
# Files generated by this notebook

# File storing normalized template experiment
normalized_template_filename = params["normalized_template_filename"]

# File storing un-normalized template experiment for us in downstream DE analysis
raw_template_filename = params["raw_template_filename"]

# File storing processed template experiment,
# after samples have been selected for comparison in DE analysis
processed_template_filename = params["processed_template_filename"]

# Output summary file
output_filename = params["output_filename"]

## Get unnormalized template experiment

In [6]:
process.fetch_template_experiment(
    normalized_compendium_filename,
    metadata_filename,
    metadata_delimiter,
    metadata_experiment_colname,
    project_id,
    metadata_sample_colname,
    scaler_transform_filename,
    raw_template_filename,
    normalized_template_filename
)

## Simulate data

In [7]:
"""# Simulate multiple experiments UPDATE COMMENT
# This step creates the following files in "<local_dir>/pseudo_experiment/" directory:
#   - selected_simulated_data_SRP012656_<n>.txt
#   - selected_simulated_encoded_data_SRP012656_<n>.txt
#   - template_normalized_data_SRP012656_test.txt
# in which "<n>" is an integer in the range of [0, num_runs-1]

# REMOVE LATER
NN_architecture = "NN_test"
dataset_name = "new_model_seen_template"
# Load pickled file
scaler = pickle.load(open(scaler_filename, "rb"))

# Update simulated dir
os.makedirs(os.path.join(local_dir, "pseudo_experiment"), exist_ok=True)

#------------
simulate_expression_data.shift_template_experiment(
    normalized_compendium_filename,
    NN_architecture,
    latent_dim,
    dataset_name,
    scaler,
    experiment_to_sample_filename,
    metadata_delimiter,
    experiment_id_colname,
    sample_id_colname,
    project_id,
    local_dir,
    base_dir,
    num_runs,
)"""

'# Simulate multiple experiments UPDATE COMMENT\n# This step creates the following files in "<local_dir>/pseudo_experiment/" directory:\n#   - selected_simulated_data_SRP012656_<n>.txt\n#   - selected_simulated_encoded_data_SRP012656_<n>.txt\n#   - template_normalized_data_SRP012656_test.txt\n# in which "<n>" is an integer in the range of [0, num_runs-1]\n\n# REMOVE LATER\nNN_architecture = "NN_test"\ndataset_name = "new_model_seen_template"\n# Load pickled file\nscaler = pickle.load(open(scaler_filename, "rb"))\n\n# Update simulated dir\nos.makedirs(os.path.join(local_dir, "pseudo_experiment"), exist_ok=True)\n\n#------------\nsimulate_expression_data.shift_template_experiment(\n    normalized_compendium_filename,\n    NN_architecture,\n    latent_dim,\n    dataset_name,\n    scaler,\n    experiment_to_sample_filename,\n    metadata_delimiter,\n    experiment_id_colname,\n    sample_id_colname,\n    project_id,\n    local_dir,\n    base_dir,\n    num_runs,\n)'

## Process template and simulated experiments

* Remove samples not required for comparison
* Make sure ordering of samples matches metadata for proper comparison
* Make sure values are cast as integers if using DESeq
* Filter lowly expressed genes if using DESeq

In [8]:
# Update simulated dir
if not os.path.exists(template_process_samples_filename):
    template_process_samples_filename = None

if de_method == "deseq":
    # Process template data
    stats.process_samples_for_DESeq(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        count_threshold,
        template_process_samples_filename,
    )

    # Process simulated data
    for i in range(num_simulated):
        simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}.txt",
        )
        out_simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}_processed.txt",
        )
        stats.process_samples_for_DESeq(
            simulated_filename,
            template_DE_grouping_filename,
            out_simulated_filename,
            count_threshold,
            template_process_samples_filename,
        )
else:
    stats.process_samples_for_limma(
        raw_template_filename,
        template_DE_grouping_filename,
        processed_template_filename,
        template_process_samples_filename,
    )

    for i in range(num_simulated):
        simulated_filename = os.path.join(
            local_dir,
            "pseudo_experiment",
            f"selected_simulated_data_{project_id}_{i}.txt",
        )
        stats.process_samples_for_limma(
            simulated_filename,
            template_DE_grouping_filename,
            None,
            template_process_samples_filename,
        )

sample ids are ordered correctly
sample ids are ordered correctly
sample ids are ordered correctly


## Differential expression analysis

In [9]:
# Create subdirectory: "<local_dir>/DE_stats/"
os.makedirs(os.path.join(local_dir, "DE_stats"), exist_ok=True)

In [10]:
# Pass simulated dir to R scripts below

In [11]:
%%R -i template_DE_grouping_filename -i project_id -i processed_template_filename -i local_dir -i base_dir -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# File created: "<local_dir>/DE_stats/DE_stats_template_data_<project_id>_real.txt"
if (de_method == "deseq"){
    get_DE_stats_DESeq(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}
else{
    get_DE_stats_limma(
        template_DE_grouping_filename,
        project_id,
        processed_template_filename,
        "template",
        local_dir,
        "real"
    )
}

R[write to console]: Loading required package: S4Vectors

R[write to console]: Loading required package: stats4

R[write to console]: Loading required package: BiocGenerics

R[write to console]: Loading required package: parallel

R[write to console]: 
Attaching package: ‘BiocGenerics’


R[write to console]: The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


R[write to console]: The following object is masked from ‘package:limma’:

    plotMA


R[write to console]: The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


R[write to console]: The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply

[1] "Checking sample ordering..."
[1] TRUE


In [12]:
%%R -i template_DE_grouping_filename -i project_id -i base_dir -i local_dir -i num_simulated -i de_method

source(paste0(base_dir, '/sophie/DE_analysis.R'))

# Files created: "<local_dir>/DE_stats/DE_stats_simulated_data_<project_id>_<n>.txt"
for (i in 0:(num_simulated-1)){
    simulated_data_filename <- paste(
        local_dir,
        "pseudo_experiment/selected_simulated_data_",
        project_id,
        "_",
        i,
        "_processed.txt",
        sep = ""
    )
    if (de_method == "deseq"){
        get_DE_stats_DESeq(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
    }
    else {
        get_DE_stats_limma(
            template_DE_grouping_filename,
            project_id,
            simulated_data_filename,
            "simulated",
            local_dir,
            i
            )
        }
    }

[1] "Checking sample ordering..."
[1] TRUE
[1] "Checking sample ordering..."
[1] TRUE


## Rank genes

Genes are ranked by their "generic-ness" - how frequently these genes are changed across the simulated experiments using user-specific test statistic provided in the `col_to_rank_genes` params (i.e. log2 fold change).

In [13]:
analysis_type = "DE"
template_DE_stats_filename = os.path.join(
    local_dir, "DE_stats", f"DE_stats_template_data_{project_id}_real.txt"
)

# Added
if de_method == "deseq":
    logFC_name = "log2FoldChange"
    pvalue_name = "padj"
else:
    logFC_name = "logFC"
    pvalue_name = "adj.P.Val"

template_DE_stats, simulated_DE_summary_stats = ranking.process_and_rank_genes_pathways(
    template_DE_stats_filename,
    local_dir,
    num_simulated,
    project_id,
    analysis_type,
    col_to_rank_genes,
    logFC_name,
    pvalue_name,
)

/home/alexandra/anaconda3/envs/sophie/lib/python3.7/site-packages/pandas/core/reshape/merge.py:544: UserWarning: merging between different levels can give an unintended result (2 levels on the left, 1 on the right)
  warnings.warn(msg, UserWarning)


## Summary table

* Gene ID: Gene identifier (hgnc symbols for human data or PA number for *P. aeruginosa* data)
* (Real): Statistics for template experiment
* (Simulated): Statistics across simulated experiments
* Number of experiments: Number of simulated experiments
* Z-score: High z-score indicates that gene is more changed in template compared to the null set of simulated experiments (high z-score = highly specific to template experiment)
* Percentile (simulated): percentile rank of the median(abs(log fold change)). So its the median absolute change for that gene across the 25 simulated experiments that is then converted to a percentile rank from 0 - 100. Where a higher percentile indicates that the gene was highly changed frequently and would suggest that the gene is more commonly DE.
* Percent DE (simulated): the fraction of the simulated experiments in which that gene was found to be DE using (log fold change > 1 and adjusted p-value < 0.05). _Note:_ you may find that many genes have a 0 fraction. This is because there is some compression that happens when pushing data through the VAE so the variance of the simulated experiments is lower compared to the real experiment. We are aware of this limitation in the VAE and are looking at how to improve the variance and biological signal captured by the VAE, however we were still able to demonstrate that for now the VAE is able to simulate realistic looking biological experiments in our previous [paper](https://academic.oup.com/gigascience/article/9/11/giaa117/5952607).


**Note:**
* If using DESeq, genes with NaN in only the `Adj P-value (Real)` column are those genes flagged because of the `cooksCutoff` parameter. The cook's distance as a diagnostic to tell if a single sample has a count which has a disproportionate impact on the log fold change and p-values. These genes are flagged with an NA in the pvalue and padj columns of the result table.

* If using DESeq with count threshold, some genes may not be present in all simulated experiments (i.e. the `Number of experiments (simulated)` will not equal the number of simulated experiments you specified in the beginning. This pre-filtering will lead to some genes found in few simulated experiments and so the background/null set for that gene is not robust. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

* If using DESeq without count threshold, some genes may still not be present in all simulated experiments (i.e. the `Number of experiments (simulated)`  will not equal the number of simulated experiments you specified in the beginning. If the gene is 0 expressed across all samples and thus automatically given an NA in `log fold change, adjusted p-value` columns. Thus, the user should sort by both z-score and number of experiments to identify specific expressed genes.

For more information you can read [DESeq FAQs](https://bioconductor.org/packages/release/bioc/vignettes/DESeq2/inst/doc/DESeq2.html#pvaluesNA)

In [14]:
# Get summary table
summary_gene_ranks = ranking.generate_summary_table(
    template_DE_stats_filename,
    template_DE_stats,
    simulated_DE_summary_stats,
    col_to_rank_genes,
    local_dir,
    "gene",
    params,
)

summary_gene_ranks.sort_values(by="Z score", ascending=False).head(10)

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
ENHO,ENHO,2.028109e-08,16835.0,2.743930,-2.743930,4.472785e-01,4495.0,25.336866,0.0,0.107937,0.000014,2,188598.057146
DMWD,DMWD,3.810378e-07,11146.0,0.655075,-0.655075,9.838326e-01,2.0,0.005638,0.0,0.001170,0.000023,2,28897.253777
ZNF296,ZNF296,1.277871e-02,9059.0,0.460130,0.460130,6.475260e-02,5402.0,30.450471,0.0,0.124839,0.000019,2,17795.345441
CPNE2,CPNE2,1.687814e-07,12120.0,0.768129,-0.768129,4.693755e-07,8971.0,50.572250,0.0,0.201419,0.000039,2,14449.801485
VIL1,VIL1,1.014520e-08,16767.0,2.643855,2.643855,4.840848e-04,15054.0,84.867790,0.0,0.550311,0.000156,2,13448.918133
IL4R,IL4R,1.391486e-01,7214.0,0.339368,-0.339368,8.179595e-01,102.0,0.569431,0.0,0.013540,0.000027,2,11891.831822
NAPB,NAPB,2.084979e-01,4721.0,0.215870,0.215870,3.278287e-01,3232.0,18.216158,0.0,0.086247,0.000012,2,10802.046773
AQP2,AQP2,1.984459e-05,16888.0,2.832715,2.832715,8.635008e-01,204.0,1.144500,0.0,0.019169,0.000263,2,10683.002696
ZCCHC9,ZCCHC9,1.928410e-06,10908.0,0.628055,0.628055,6.433344e-01,103.0,0.575069,0.0,0.013579,0.000061,2,10114.638257
LINC00525,LINC00525,1.215430e-02,16163.0,2.069960,2.069960,4.925465e-01,5151.0,29.035350,0.0,0.120083,0.000196,2,9961.600004


In [15]:
summary_gene_ranks.isna().any()

Gene ID                                 False
Adj P-value (Real)                       True
Rank (Real)                              True
abs(log2FoldChange) (Real)               True
log2FoldChange (Real)                    True
Median adj p-value (simulated)           True
Rank (simulated)                         True
Percentile (simulated)                   True
Percent DE (simulated)                  False
Mean abs(log2FoldChange) (simulated)     True
Std deviation (simulated)                True
Number of experiments (simulated)       False
Z score                                  True
dtype: bool

In [16]:
summary_gene_ranks[summary_gene_ranks.isna().any(axis=1)]

,Gene ID,Adj P-value (Real),Rank (Real),abs(log2FoldChange) (Real),log2FoldChange (Real),Median adj p-value (simulated),Rank (simulated),Percentile (simulated),Percent DE (simulated),Mean abs(log2FoldChange) (simulated),Std deviation (simulated),Number of experiments (simulated),Z score
ART1,ART1,2.051606e-07,16922.0,2.873521,-2.873521,1.018678e-01,15176.0,85.555618,0.0,0.568481,NaN,1,NaN
LINC01799,LINC01799,6.306450e-02,16688.0,2.552267,-2.552267,1.687554e-01,15388.0,86.750860,0.0,0.598926,NaN,1,NaN
OR5P2,OR5P2,4.586367e-03,16608.0,2.462858,-2.462858,6.159564e-01,10891.0,61.397080,0.0,0.259438,NaN,1,NaN
OR2S2,OR2S2,9.157873e-02,16454.0,2.311245,-2.311245,2.743789e-02,15922.0,89.761515,0.0,0.686876,NaN,1,NaN
OR5P3,OR5P3,4.315702e-02,16406.0,2.271398,-2.271398,6.200342e-01,8921.0,50.290353,0.0,0.199759,NaN,1,NaN
FAM170B,FAM170B,1.234430e-01,15978.0,1.956609,-1.956609,8.102610e-02,17189.0,96.904775,0.0,1.093901,NaN,1,NaN
LINC01656,LINC01656,3.773415e-01,15893.0,1.912057,-1.912057,9.076653e-01,3378.0,19.039296,0.0,0.088754,NaN,1,NaN
LINC01180,LINC01180,2.318194e-01,15843.0,1.880683,1.880683,4.593101e-01,13672.0,77.076168,0.0,0.410852,NaN,1,NaN
TAS2R50,TAS2R50,4.457160e-02,15814.0,1.861618,1.861618,8.581452e-01,1318.0,7.425156,0.0,0.050000,NaN,1,NaN
TEX13B,TEX13B,3.161243e-01,15797.0,1.844302,1.844302,7.807983e-01,2811.0,15.842589,0.0,0.078630,NaN,1,NaN


In [17]:
# Save
summary_gene_ranks.to_csv(output_filename, sep="\t")